**[Voltar para a Página Inicial do Curso](https://www.kaggle.com/c/ml-em-python)**

# **Mineração de Textos 2**
1. Problemas comuns no uso de textos (alta dimensionalidade, matrizes esparsas, etc) e como resolvê-los.
2. Extração de Tópicos 
3. Análise de sentimentos

![](https://www.kdnuggets.com/images/cartoon-machine-learning-class.jpg)
## Caso de Uso: Notícias da Folha de São Paulo


In [25]:
# Vamos usar os dados das notícias da Folha de São Paulo
import pandas as pd
noticias = pd.read_csv('../input/news-of-the-site-folhauol/articles.csv')

# O dataset completo contém notícias de Jan/15 a Set/17. Vamos filtrar para usar apenas o mês de junho/17
noticias = noticias[(noticias['date'] >= '2017-06-01') & (noticias['date'] <= '2017-06-30')]
noticias.shape

,title,text,date,category,subcategory,link
369,Prestação de contas de projeto de José de Abre...,O Ministério da Cultura (MinC) reprovou a pres...,2017-06-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
370,"Doria é dos 'piores políticos que já tivemos',...","O ex-governador de São Paulo Alberto Goldman, ...",2017-06-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
377,\n\t\tVolkswagen prepara troca de presidentes ...,EDUARDO SODRÉ COLUNISTA DA FOLHA A Volkswagen...,2017-06-10,sobretudo,rodas,http://www1.folha.uol.com.br/sobretudo/rodas/2...
378,Temer sanciona criação de fundo eleitoral e ve...,O presidente Michel Temer sancionou com vetos ...,2017-06-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
379,"Possível candidato ao Senado, Bernardinho crit...","Bernardinho vestiu uma camisa polo, mas desta ...",2017-06-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
380,Goleiro da seleção lembra de choro ao ser disp...,Confirmado como titular da seleção brasileira ...,2017-06-10,esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/10/1...
381,Latam inaugura voo direto de 15 horas entre Ch...,Animem-se os que pretendem cruzar o Pacífico e...,2017-06-10,turismo,NaN,http://www1.folha.uol.com.br/turismo/2017/10/1...
382,Entenda por que o primeiro-ministro do Japão a...,"O primeiro-ministro japonês, Shinzo Abe, decid...",2017-06-10,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/10/192...
383,"Antes de ser 'de periguete', bandagem de Léger...",O advento da Aids encaretou o mundo e a moda. ...,2017-06-10,colunas,pedrodiniz,http://www1.folha.uol.com.br/colunas/pedrodini...
384,"Após prisão de Nuzman, COB marca reunião extra...",O COB (Comitê Olímpico do Brasil) convocou uma...,2017-06-10,esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/10/1...


In [27]:
# CONFIGURACOES DO PANDAS

# Para mostrar todos os caracteres
pd.set_option('display.max_colwidth', -1)
noticias.head()

# Para voltar a truncar os caracteres a 50 caracteres
pd.set_option('display.max_colwidth', 50)
noticias.head()

,title,text,date,category,subcategory,link
369,Prestação de contas de projeto de José de Abre...,O Ministério da Cultura (MinC) reprovou a pres...,2017-06-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
370,"Doria é dos 'piores políticos que já tivemos',...","O ex-governador de São Paulo Alberto Goldman, ...",2017-06-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
377,\n\t\tVolkswagen prepara troca de presidentes ...,EDUARDO SODRÉ COLUNISTA DA FOLHA A Volkswagen...,2017-06-10,sobretudo,rodas,http://www1.folha.uol.com.br/sobretudo/rodas/2...
378,Temer sanciona criação de fundo eleitoral e ve...,O presidente Michel Temer sancionou com vetos ...,2017-06-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
379,"Possível candidato ao Senado, Bernardinho crit...","Bernardinho vestiu uma camisa polo, mas desta ...",2017-06-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...


In [29]:
# Pré-processamento. Vamos concatenar a coluna titulo e a texto
import re
notic = (noticias['title'].astype(str)) # 8 mil palavras + " " + noticias['text'].astype(str)) # 65 mil palavras
notic = [re.sub("(\\d|\\W)+", " ", e) for e in notic]

# Define as stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('portuguese')

# Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
vocab = CountVectorizer(stop_words=stopwords, lowercase=True)
vetor=vocab.fit_transform(notic) 
print(len(vocab.vocabulary_))
topicos = vocab.get_feature_names()
bow = pd.DataFrame(vetor.toarray(),columns=topicos)
bow.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
8514


,abafa,abaixo,abala,abalam,abalar,abalaram,abandonam,abandono,abastece,abaterem,abbey,abdelmassih,abdominal,abelhas,aberta,abertas,aberto,abertos,abertura,abismo,abolir,abono,aborda,abordagens,abordar,aborta,aborto,aborígenes,abraji,abraços,abre,abreu,abreviar,abreviaria,abrigo,abril,abrir,abscesso,absoluta,absolve,...,árabe,árabes,árbitro,área,áreas,árvore,árvores,áudio,áudios,áustria,ânimo,élysées,épica,épico,época,ética,ícone,ídolo,ímã,índia,índice,índios,íntegra,óbitos,ócio,ódio,óleo,ópera,órama,órgão,órgãos,ótima,ônibus,última,últimas,último,úmido,única,único,útil
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 1. Problemas Comuns no Uso de Textos e Como Resolver
* Alta dimensionaldade
* Matrizes Esparsas

## Alta Dimensionalidade
**Dimensionalidade** se refere à quantidade de variáveis ou atributos (colunas) de um dataset. A alta dimensionalidade acontece quando a quantidade de dimensões (d) é maior do que a quantidade de observações (n). Podemos chegar o tamanho com o `shape`. Em casos em que a quantidade de dimensões é expressivamente maior, o processamento destes dados se torna demorado e requer mais recursos de infraestrutura.

In [30]:
# Bag of Words - Com Alta Dimensionalidade
bow.shape

# Dataset de Noticias - Não tem Alta Dimensionalidade
noticias.shape

# Imprime o formato da matriz
import numpy as np
np.shape(vetor)

(3735, 8514)

## Como resolver o problema da alta dimensionalidade
Ter muitas dimensões pode causar overfitting. Para mitigar o problema da alta dimensionalidade, devemos diminuir a quantidade de dimensões, selecionar menos features, selecionar as features mais importantes, chamamos isso de **feature selection**. São exemplos de técnicas seleção de variáveis: 
1. Descartar variáveis que contém em sua **maioria valores ausentes** (NaN)
2. Descartar variáveis que tem **baixa variância**. Por exemplo, todos os dados de UF são iguais a SP. 
3. Descartar uma das variáveis que são altamente correlacionadas com outras. Se usar uma variável ou outra teria o resultado parecido. Então descartamos uma delas.
4. Visualizar as **variáveis mais importantes** e descartar as de menor importância. Ex: Random Forest.
5. Eliminar variáveis que não agregam na **performance do modelo**. Para descobrir quais são, podemos usar o **Backward Feature Elimination** ou **Foward Feature Elimination**. No Backward, treinamos o modelo várias vezes começando com todas as variáveis e, em cada iteração, remover uma das variáveis para descobrir qual das variáveis tem menor impacto no resultado do modelo. Já no Foward, fazemos o contrário, começando com apenas uma variável e vamos aumentando a seleção de variáveis em cada iteração e avaliamos quais os impactos de cada configuraçao. Por fim, remover as variáveis que não agregam tanto no modelo.

Outra alternativa é utilizar técnicas avançadas de extração de variáveis. A **Feature Extraction** busca descobrir o menor conjunto de variáveis, que combinadas, contém as informações mais relevantes que são aquelas que mais explicam a variancia dos dados. Como Técnicas Avançadas de feature extraction, temos, por exemplo: 
* PCA - Principal Component Analysis: 
* LDA - Linear Discriminant Analysis
* t-SNE - t-Distributed Stochastic Neighbor Embedding
* LDA - Linear Discriminant Analysis
* SVD - Singular Value Decomposition

## Matrizes Esparsas
Matrizes que contém a maior parte de valores iguais a zero são chamadas de **matrizes esparsas**. Diferentes de **matrizes densas** em que a maioria dos valores são diferentes de zero. Matrizes esparsas são comuns na mineração de textos. Porém, é um problema porque demanda alto poder computacional ou alto custo de infraestrutura para processamento destas matrizes. Saber lidar com matrizes esparsas pode salvar muito tempo e recursos computacionais de infraestrutura. Quanto maior a matriz, maior a complexidade. Uma matriz é considerada como esparsa ou densa através do cálculo da proporção de elementos iguais a 0 divididos pela quantidade total de elementos. O ideal é que se tenha a menor esparsidade possível para economizar recursos. Uma matriz é esparsa, quando sua esparsidade é maior que 50%.

`esparsidade = qtd elementos iguais a 0 / qtd total de elementos`

In [31]:
# Checa a esparsidade da matriz. Percentual de celulas que contém zeros
print("Esparsidade: ", ((vetor.todense() == 0).sum()/vetor.todense().size)*100, "%")

Esparsidade:  99.91868185293048 %


## Como resolver o problema da matriz esparsa
Para lidar com matrizes esparsas, usamos uma estrutura de dados alternativa que usa técnicas de álgebra linear. Nesta estrutura, os valores 0 são ignorados e apenas os valores diferentes de zero são armazenados. Assim, fazemos a **compressão de linhas esparsas** ou Compressed Sparse Row - **CSR** através da função `csr_matriz()` do SciPy.

In [32]:
# Importa as bibliotecas
from numpy import array
from scipy.sparse import csr_matrix

# Cria uma matriz de exemplo
A = array([[1, 0, 0, 1, 0, 0], [0, 0, 2, 0, 0, 1], [0, 0, 0, 2, 0, 0]])
print(A)

# Calcula a esparsidade
print("Esparsidade: ", ((A == 0).sum()/A.size)*100, "%")

[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]
Esparsidade:  72.22222222222221 %


In [7]:
# convert to sparse matrix (CSR method)
# formato: (linha,coluna) valor
S = csr_matrix(A)
print(S)

  (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 5)	1
  (2, 3)	2


# 2. Extração de Tópicos
A extração de tópicos é útil para identificar qual o assunto do texto. O método **LDA - Latent Dirichlet Allocation** é um dos mais usados. O LDA é um modelo de aprendizado **não-supervisionado** que recebe documentos como entrada e identifica os tópicos como saída. O modelo também diz em que percentual o documento diz sobre cada tópico. Utiliza a probabilidade de uma palavra aparecer no tópico ou no documento.
![](https://cdn-images-1.medium.com/max/800/1*RcyXi1lMESMq3X_RC7bL0Q.png)
![](https://cdn-images-1.medium.com/max/1200/1*GpPEknlKMQ09UkEtY6MZOw.png)
Como parâmetro podemos definir o número de tópicos, a quantidade de palavras por tópico e o número de tópicos por documento. O LDA recebe como entrada a bag of words.

In [37]:
# Cria o LDA
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=3, max_iter=10).fit(vetor)

In [35]:
# Avaliação do Score. Quanto maior, melhor
print(lda.score(vetor))

# Avaliação da perplexidade. Quanto menor, melhor
print(lda.perplexity(vetor))

6925.200009549216


In [36]:
# Mostra o Resultado
def resultado(modelo, feature_names, qtd_topicos):
    for topic_idx, topic in enumerate(modelo.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-qtd_topicos - 1:-1]]))

resultado(lda, topicos, 10) # 10 estabelece o numero de palavras que aparecem

Topic 0:
diz temer após vai governo sp sobre paulo anos stf
Topic 1:
diz nova trump após eua brasil copa melhor governo sp
Topic 2:
temer diz contra sobre sp presidente brasil jato denúncia lava


In [38]:
# Visualização Gráfica
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
vis=sklearn_lda.prepare(lda, vetor, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


## Vantagens e Desvantagens do LDA
O LDA é geralmente rápido, mas pode ser bem demorado se aplicado em grandes documentos, em muitos documentos ou em situações com vastos vocabulários. É fácil interpretar a saída porque possui uma abordagem bem intuitiva, porém depende muito da interpretação humana. Como os tópicos são identificados pelo computador numa abordagem não-supervisionada, não podemos influenciar na saída de quais tópicos são gerados. Neste caso, a saída para melhorar o resultado é caprichar no pré-processamento do texto antes do uso do LDA. O pré-processamento pode incluir uma ou mais técnicas como: a remoção de caracteres especiais e números, remoção de stopwords, remoção de palavras raras que acontecem em poucos documentos e aplicação de stemming.

### Agora se você quiser gastar um pouquinho ...
* Ferramenta Google: https://cloud.google.com/natural-language/
* Ferramenta Microsoft: https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/text-analytics-user-scenarios
* MeaningCloud: https://www.meaningcloud.com/developer/topics-extraction


# 3. Análise de Sentimentos
Como as pessoas se sentem sobre determinado produto, notícia ou filme? Também chamado de mineração de opinião, a **Análise de Sentimentos** busca descobrir a opinião das pessoas, utilizando o texto de suas avaliações, comentários ou postagens em redes sociais. 

![](https://cdn-images-1.medium.com/max/1600/1*FRd4BsrZ2VxKLbvVYJQC6w.png)

Podemos também aprofundar o nível da análise e ao invés de analisar a **polaridade** do sentimento (positivo, negativo), podemos analisar as **emoções**.
![](https://img.elo7.com.br/product/main/19C5C37/cartoes-de-emocoes-decoracao-infantilx-decoracao-para-bebex-decoracao-para-quarto-de-crianx-decoraca.jpg)

## Tipos
* Lexicon 
* Machine Learning

# Lexicon
Consiste em um banco de dados de palavras já classificadas como positivas, neutras ou negativas. Conta a quantidade de vezes em que as palavras positivas e negativas aparecem. O problema é que não tem capacidade de entender o contexto, sarcarsmo, variações das palavras conhecidas. Ou seja, pode errar muito. Usaremos o Polyglot porque consegue entender nosso idioma **Português**, dentre vários outros idiomas.

Polyglot: https://polyglot.readthedocs.io/en/latest/Sentiment.html

In [12]:
! pip install pyicu 
! pip install pycld2
! pip install morfessor
! polyglot download embeddings2.en
! polyglot download embeddings2.pt
! polyglot download sentiment2.sk
! polyglot download sentiment2.en # Faz o download do lexicon em English
! polyglot download sentiment2.pt # Faz o download do lexicon em Portugues
import polyglot
from polyglot.text import Text, Word

    100% |████████████████████████████████| 215kB 9.4MB/s eta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/3f/45/7e/ccee9f1fe52787595e92641b5645cdf2cb40096749b39b4422
Successfully built pyicu
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 14.3MB 3.1MB/s eta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/7b/44/44/ec4c5e25e095f02aa0e63ef2bf0cc8badda5877330ffa5fbe4
Successfully built pycld2
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Stored in directory: /tmp/.cache/pip/wheels/43/74/e2/92a807d8662f5a0df908ba20887f8ad02bf1b130d3939b65b7
Successfully built morfessor
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[polyglot_data] Downloa

In [13]:
# Polaridade das Palavras: +1 positivo, 0 neutro, -1 negativo
text = Text("O filme foi bom, mas a legenda estava ruim")

print("{:<16}{}".format("Word", "Polarity")+"\n"+"-"*30)
for w in text.words:
    print("{:<16}{:>2}".format(w, w.polarity))

Word            Polarity
------------------------------
O                0
filme            0
foi              0
bom              1
,                0
mas              0
a                0
legenda          0
estava           0
ruim            -1


In [14]:
# Baixa pacotes.
! polyglot download sentiment2.la
! polyglot download sentiment2.gl
! polyglot download sentiment2.lv
! polyglot download sentiment2.it
! polyglot download sentiment2.ca
! polyglot download sentiment2.es
! polyglot download sentiment2.war
! polyglot download sentiment2.ht
! polyglot download sentiment2.lt
! polyglot download sentiment2.tk
! polyglot download sentiment2.ga
! polyglot download sentiment2.fi

[polyglot_data] Downloading package sentiment2.la to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.gl to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.lv to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.it to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.ca to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.es to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.war to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.ht to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.lt to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.tk to
[polyglot_data]     /tmp/polyglot_data...
[polyglot_data] Downloading package sentiment2.ga

In [41]:
# Apenas para a primeira notícia da folha de sao paulo
Text(noticias.iloc[0,0]).polarity
    
# Cria um loop para calcular a polaridade das 100 primeiras noticias
noticias['polaridade'] = 0

#for w in range(len(noticias)):    # Para todas as noticias        
for w in range(100):               # Apenas para as 100 primeiras noticias
    try:
        noticias.iloc[w,6] = Text(noticias.iloc[w,0]).polarity
    except ZeroDivisionError:
        noticias.iloc[w,6] = 0

# Polaridade das Noticias em Geral
noticias.polaridade.describe()

# Polaridade por Categoria
noticias.groupby('category')['polaridade'].describe()

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


,count,mean,std,min,25%,50%,75%,max
category,,,,,,,,
ambiente,26.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
asmais,2.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
banco-de-dados,4.0,-0.250000,0.500000,-1.0,-0.25,0.0,0.0,0.000000
ciencia,38.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
colunas,659.0,-0.002529,0.100204,-1.0,0.00,0.0,0.0,1.000000
comida,1.0,0.000000,NaN,0.0,0.00,0.0,0.0,0.000000
cotidiano,385.0,-0.015152,0.117180,-1.0,0.00,0.0,0.0,0.000000
educacao,26.0,-0.038462,0.196116,-1.0,0.00,0.0,0.0,0.000000
empreendedorsocial,30.0,0.033333,0.182574,0.0,0.00,0.0,0.0,1.000000


Também existe o **VADER** do NLTK no Python. O Vader é muito usado para pequenos textos, como tweets, avaliações e comentários em redes sociais. Mede a polaridade de palavras e também considera emoticons.

# Usando Machine Learning para Análise de Sentimentos
Usamos classificação para fazer a análise de sentimentos de uma forma automática. É mais fácil de escalar, mais assertiva, porém requer dados de treino.
![](https://content-static.upwork.com/blog/uploads/sites/3/2017/06/27091802/image-8.png)

Teste na Web: https://text-processing.com/demo/sentiment/

## Vamos fazer análise de sentimentos de opiniões sobre filmes

In [16]:
# Importa os dados, mostra a proporção de sentimentos e imprime as primeiras linhas
import pandas as pd
train=pd.read_csv('../input/bag-of-words-meets-bags-of-popcorn-/labeledTrainData.tsv',delimiter='\t',quoting=3)
test=pd.read_csv('../input/bag-of-words-meets-bags-of-popcorn-/testData.tsv',delimiter='\t',quoting=3)

print(train.sentiment.value_counts())
train.head(4)

1    12500
0    12500
Name: sentiment, dtype: int64


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."


In [17]:
# Analisa o primeiro comentário
train["review"][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [44]:
# Vamos diminuir os datasets para 1.000 linhas cada pra ser mais rapido
train=train[0:1000]
test=test[0:1000]

In [19]:
# Pre-processamento
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from string import punctuation
stopwords=set(stopwords.words('english')+list(punctuation))

def preprocessamento(review):
    # Remove as tags HTML
    review = BeautifulSoup(review,'html.parser').get_text()
    # Remove o for diferente de letras
    review = re.sub("[^a-zA-Z]", " ", review)
    # Transforma para letra minuscula
    review = review.lower()
    # Faz a tokenizacao, separacao em palavras
    token = nltk.word_tokenize(review)
    # Remove Stopwords
    token = [w for w in token if w not in stopwords]
    # Stemming
    review = [nltk.stem.SnowballStemmer('english').stem(w) for w in token]
    # Junta as palavras de uma opinião em uma string separadas por espaçoe
    return " ".join(review)

treino = train.review.apply(preprocessamento)
teste = test.review.apply(preprocessamento)

In [20]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import words
vetor = TfidfVectorizer(max_features=40000)
vetortreino = vetor.fit_transform(treino)
vetorteste = vetor.transform(teste) 
vocab = vetor.get_feature_names()
print(vocab[:50])

['aag', 'abandon', 'abbey', 'abbi', 'abbott', 'abc', 'abdic', 'abduct', 'abducte', 'abel', 'abhay', 'abhorr', 'abigail', 'abil', 'abl', 'abli', 'abnorm', 'abod', 'abomin', 'aborigin', 'abort', 'abound', 'abraham', 'abroad', 'abrupt', 'absenc', 'absent', 'absolut', 'absolutley', 'absorb', 'abstract', 'absurd', 'abund', 'abus', 'abysm', 'abyss', 'academ', 'academi', 'acceler', 'accent', 'accentu', 'accept', 'acces', 'access', 'accid', 'accident', 'acclaim', 'accolad', 'accommod', 'accompani']


In [21]:
# Modelo SVM
from sklearn import metrics
from sklearn.svm import LinearSVC
svm = LinearSVC(random_state=1)
svm.fit(vetortreino, train.sentiment)
previsaosvm = svm.predict(vetorteste)
metrics.accuracy_score(train.sentiment, previsaosvm)

0.506

In [22]:
# Modelo Random Forest
from sklearn.ensemble import RandomForestClassifier
modelorf=RandomForestClassifier(random_state=1, n_estimators=100)
modelorf.fit(vetortreino, train.sentiment)
previsaorf=modelorf.predict(vetorteste)
metrics.accuracy_score(train.sentiment, previsaorf)

0.511

## EXTRA!!! Grid Search + Cross Validation

In [23]:
# Random Forest com GridSearch
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[10,100,200],'criterion':['entropy','gini'],
               'max_depth':[5,7,9,11]}
grid_search = GridSearchCV(estimator = modelorf,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 10,
                            n_jobs = -1)
grid_search = grid_search.fit(vetortreino, train.sentiment)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print('Melhor Resultado :',best_accuracy)
print('Melhores Parâmetros:\n',best_parameters)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Melhor Resultado : 0.802
Melhores Parâmetros:
 {'criterion': 'entropy', 'max_depth': 11, 'n_estimators': 200}


--- 
## Material Complementar
1. [Livro online de NLTK em Python](http://www.nltk.org/book/)

---
### Próxima Aula
[10. Raspagem de Sites](https://www.kaggle.com/debkings/10-raspagem-de-sites)